In [1]:
#import packages
import pandas as pd
import re
import numpy as np

In [2]:
df=pd.read_csv('Data/gendered_titles_final.csv')
df=df[['Name','Text']]

In [3]:
df['TextShort']=df['Text'].str[:1000]
df['Year8']= df['TextShort'].apply(lambda x: re.findall(r'\b\d{8}\b', str(x)))
df['Year4'] = df['TextShort'].apply(lambda x: re.findall(r'\b\d{4}\b', str(x)))
df['Year BC/AD/CE'] = df['TextShort'].apply(lambda x: re.findall(r'\b\d{3}\b', str(x)))
df['BC']=df['TextShort'].apply(lambda x: re.findall(r' bc ', str(x)))
df['AD']=df['TextShort'].apply(lambda x: re.findall(r' ad ', str(x)))
df['CE']=df['TextShort'].apply(lambda x: re.findall(r' ce ', str(x)))
df['BC']=df['BC'].apply(lambda x: 1 if len(x)>0 else 0)
df['AD']=df['AD'].apply(lambda x: 1 if len(x)>0 else 0)
df['CE']=df['CE'].apply(lambda x: 1 if len(x)>0 else 0)

In [4]:
def determine_birth_year(row):
        if row['BC']==1 and len(row['Year BC/AD/CE']) > 0:
            return "-"+row['Year BC/AD/CE'][0]
        elif (row['AD']==1 or row['CE']==1) and len(row['Year BC/AD/CE'])>0:
            return row['Year BC/AD/CE'][0]
        elif len(row['Year8'])>0:
            return row['Year8'][0][0:4]
        elif len(row['Year4']) > 0:
            return row['Year4'][0]
        else:
            return np.nan

df['Birth year'] = df.apply(determine_birth_year, axis=1)
df['Birth year'] = pd.to_numeric(df['Birth year'], errors='coerce')
df=df[df['Birth year'].notnull()]
df=df[df['Birth year']<=2024]


In [5]:
df=df[['Name','Birth year']]
df.to_csv('Data/estimated_birth_years_1000.csv',index=False)
